In [1]:
pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import simpy
from scipy import stats 
from random import expovariate,seed
import matplotlib.pyplot as pp
import numpy as np

%matplotlib inline

# Total de carriles
NUM_CARRILES = 3
# Intervalo de tiempo en que llegan vehiculos (minutos)
INTERVALO_LLEGADA = 3
# Tiempo maximo de estacionamiento
# Tiempo de simulación
TIEMPO_SIMULACION = 1800
MEDIA_REVISION = 10

In [19]:
import random
tiempo = []
class EMOV(object):
  def __init__(self, environment, num_carriles):
    self.env = environment
    self.tipo = simpy.Resource(environment, num_carriles)
  
  def estacionamiento(self):
    # aqui esta el tiempo que se necesita para atender un cliente con 
    #la distribucion exponencial y la media dada de 3 minutos
    yield self.env.timeout(expovariate(1.0/random.randint(3,5)))

  def turno(self,tipo):
    # aqui esta el tiempo que se necesita para atender un cliente con 
    #la distribucion exponencial y la media dada de 3 minutos
    yield self.env.timeout(expovariate(1.0/random.randint(1,3)))

  def pago(self,tipo):
    # aqui esta el tiempo que se necesita para atender un cliente con 
    #la distribucion exponencial y la media dada de 3 minutos
    yield self.env.timeout(expovariate(1.0/random.randint(5,10)))

  def atender_vehiculo(self,num_carril):
    yield self.env.timeout(expovariate(1.0/10+np.std(MEDIA_REVISION)))


def revision(env, tipo ,carro):
  llegada = round(env.now,2)  
  print('Llega vehiculo a revision: %s a la hora %.2f.' % (tipo, env.now))
  with carro.tipo.request() as vehiculo:
    yield vehiculo
    yield env.process(carro.estacionamiento())
    yield env.process(carro.turno(tipo))
    yield env.process(carro.pago(tipo))
    yield env.process(carro.atender_vehiculo(random.randint(1,2)))
    final = round(env.now,2)  
    tiempo.append(final-llegada)
    print('vehiculo [%s] fue atendido a las %.2f.'%(carro, env.now))
    

def ejecutar_simulacion(env, num_carril):
  vehiculo = EMOV(env, num_carril)
  for i in range(5):
    env.process(revision(env, 'vehiculo-%d'%(i+1),vehiculo))
  while True:
    yield env.timeout(random.randint(5, 10))
    i+=1
    env.process(revision(env, 'vehiculo-%d'%(i+1), vehiculo))
random.seed(77)
env=simpy.Environment()
# Aqui estoy mandando a ejecutar la simulacion con la cantidad de cajeros
# previamente definida
env.process(ejecutar_simulacion(env, NUM_CARRILES))
env.run(until = TIEMPO_SIMULACION)
        

Llega vehiculo a revision: vehiculo-1 a la hora 0.00.
Llega vehiculo a revision: vehiculo-2 a la hora 0.00.
Llega vehiculo a revision: vehiculo-3 a la hora 0.00.
Llega vehiculo a revision: vehiculo-4 a la hora 0.00.
Llega vehiculo a revision: vehiculo-5 a la hora 0.00.
Llega vehiculo a revision: vehiculo-6 a la hora 7.00.
Llega vehiculo a revision: vehiculo-7 a la hora 12.00.
vehiculo [<__main__.EMOV object at 0x7f22e6c08890>] fue atendido a las 15.69.
vehiculo [<__main__.EMOV object at 0x7f22e6c08890>] fue atendido a las 16.04.
Llega vehiculo a revision: vehiculo-8 a la hora 20.00.
vehiculo [<__main__.EMOV object at 0x7f22e6c08890>] fue atendido a las 25.11.
Llega vehiculo a revision: vehiculo-9 a la hora 29.00.
Llega vehiculo a revision: vehiculo-10 a la hora 36.00.
vehiculo [<__main__.EMOV object at 0x7f22e6c08890>] fue atendido a las 38.32.
Llega vehiculo a revision: vehiculo-11 a la hora 44.00.
Llega vehiculo a revision: vehiculo-12 a la hora 49.00.
vehiculo [<__main__.EMOV object